In [1]:
import pandas as pd

BASE_PATH = "/home/jiwon/data/movies/dailyboxoffice"
df = pd.read_parquet(BASE_PATH)

In [3]:
df.columns

Index(['rnum', 'rank', 'rankInten', 'rankOldAndNew', 'movieCd', 'movieNm',
       'openDt', 'salesAmt', 'salesShare', 'salesInten', 'salesChange',
       'salesAcc', 'audiCnt', 'audiInten', 'audiChange', 'audiAcc', 'scrnCnt',
       'showCnt', 'dt', 'multiMovieYn', 'repNationCd'],
      dtype='object')

partition => 값이 디렉토리 컬럼 이름으로 대체되면서 df에서는 삭제됨 
```python
jiwon@JIWON  ~/temp/test_p_1  tree
.
├── openDt=1973-01-13
│   └── movieCd=19860125
│       └── 735f93fa40504ed999fa18ee0f11b89d-0.parquet
```

In [4]:
df.to_parquet("~/temp/test_p_1", partition_cols=['openDt','movieCd'])

## 20250319 QUIZ 1

- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.combine_first.html#pandas.DataFrame.combine_first

In [21]:
A = pd.DataFrame({
    "movieCd": ["1001", "1002", "1003"],
    "multiMovieYn": ["Y", "Y", "N"],
    "repNationCd": ["K", "F", None]
})

print(A)
# movieCd multiMovieYn repNationCd
# 0    1001            Y           K
# 1    1002            Y           F
# 2    1003            N        None

  movieCd multiMovieYn repNationCd
0    1001            Y           K
1    1002            Y           F
2    1003            N        None


In [22]:
B = pd.DataFrame({
    "movieCd": ["1001", "1003", "1004"],
    "multiMovieYn": [None, "Y", "Y"],
    "repNationCd": [None, "F", "K"]
})

print(B)
#   movieCd multiMovieYn repNationCd
# 0    1001         None        None
# 1    1003            Y           F
# 2    1004            Y           K


  movieCd multiMovieYn repNationCd
0    1001         None        None
1    1003            Y           F
2    1004            Y           K


In [25]:
print(A_set)

        multiMovieYn repNationCd
movieCd                         
1001               Y           K
1002               Y           F
1003               N        None


In [26]:
print(B_set)

        multiMovieYn repNationCd
movieCd                         
1001            None        None
1003               Y           F
1004               Y           K


In [30]:
# TODO
# comdine 사용!

A_set = A.set_index("movieCd")
B_set = B.set_index("movieCd")

r_df = A_set.combine_first(B_set)
r_df = r_df.reset_index()

print(r_df)

# movieCd multiMovieYn repNationCd
# 0    1001            Y           K
# 1    1002            Y           F
# 2    1003            N           F
# 3    1004            Y           K

  movieCd multiMovieYn repNationCd
0    1001            Y           K
1    1002            Y           F
2    1003            N           F
3    1004            Y           K


## 추가 QUIZ 2

- https://pandas.pydata.org/docs/reference/api/pandas.Series.combine_first.html#pandas.Series.combine_first

In [33]:
merged_df = A.merge(B, on="movieCd", how="outer", suffixes=("_A", "_B"))
print(merged_df)
#  movieCd multiMovieYn_A repNationCd_A multiMovieYn_B repNationCd_B
# 0    1001              Y             K           None          None
# 1    1002              Y             F            NaN           NaN
# 2    1003              N          None              Y             F
# 3    1004            NaN           NaN              Y             K

  movieCd multiMovieYn_A repNationCd_A multiMovieYn_B repNationCd_B
0    1001              Y             K           None          None
1    1002              Y             F            NaN           NaN
2    1003              N          None              Y             F
3    1004            NaN           NaN              Y             K


In [35]:
merged_df['multiMovieYn'] = merged_df['multiMovieYn_A'].combine_first(merged_df['multiMovieYn_B'])
merged_df['repNationCd'] = merged_df['repNationCd_A'].combine_first(merged_df['repNationCd_B'])

r_df2 = merged_df[['movieCd', 'multiMovieYn', 'repNationCd']]

print(r_df2)

  movieCd multiMovieYn repNationCd
0    1001            Y           K
1    1002            Y           F
2    1003            N           F
3    1004            Y           K


In [36]:
import pandas as pd
from datetime import datetime, timedelta

In [38]:
import pandas as pd
import os


def read_df(parquet_path: str) -> pd.DataFrame:
    if os.path.exists(parquet_path):
        df = pd.read_parquet(parquet_path)
    else:
        df = None
        
    return df

In [49]:
PATH = os.path.expanduser("~/data/movies/merge/dailyboxoffice/dt=")
ds_nodash = "20240102"
# 문자열을 datetime 객체로 변환
date_obj = datetime.strptime(ds_nodash, "%Y%m%d")
# 하루 전 날짜 계산
prev_date_obj = date_obj - timedelta(days=1)
# 결과를 다시 문자열로 변환 (예: '20231231')
prev_dt = prev_date_obj.strftime("%Y%m%d")

print(f"{PATH}{prev_dt}")
previous_df = read_df(f"{PATH}{prev_dt}")
current_df = read_df(f"{PATH}{ds_nodash}")

/home/jiwon/data/movies/merge/dailyboxoffice/dt=20240101


In [50]:
type(current_df)

pandas.core.frame.DataFrame